In [1]:
import csv
import numpy as np
import cv2

import keras
import keras.models as models

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import BatchNormalization,Input, Lambda
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping

import sklearn.metrics as metrics

import json

Using TensorFlow backend.


In [2]:
target_im_wide = 320 #200
target_im_height = 160 #66
num_imgs = 8380*2 #8036*2 # *4 because flip, left, and right

steer_ofst = 0.2

prefix = '.\\my_test_data\\' #.\\data\\'
steer_angle = np.ndarray(num_imgs, dtype=np.double)
imgs = np.ndarray((num_imgs, target_im_height, target_im_wide, 3), dtype=np.uint8)

def getimage(fn_img):
    img = cv2.imread(fn_img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img = cv2.resize(img, (target_im_wide, target_im_height)) 
    return img

with open('.\my_test_data\driving_log.csv', 'r') as csvfile:
    i=0
    reader = csv.DictReader(csvfile)
    for row in reader:
        # original image and data
        steer_angle[i] = float(row['steering'])
        imgs[i] = getimage(prefix + row['center'])
        i+=1
        
        # flipped image and data
        steer_angle[i] = -1.0 * float(row['steering'])
        imgs[i] = cv2.flip(getimage(prefix + row['center']), 1)
        i+=1
        
        # left image (**assumes left steer is positive)
#        steer_angle[i] = float(row['steering']) + steer_ofst
#        fn_img=prefix + row['left']
#        fn_img = fn_img.replace(" ", "") #poorly formatted Udacity data
#        imgs[i] = getimage(fn_img)
#        i+=1 
        
        # right image (**assumes left steer is positive)
#        steer_angle[i] = float(row['steering']) - steer_ofst
#        fn_img=prefix + row['right']
#        fn_img = fn_img.replace(" ", "") #poorly formatted Udacity data
#        imgs[i] = getimage(fn_img)
#        i+=1

#imgs_swap = np.moveaxis(imgs, -1, 1)
print(steer_angle.shape)  # steering angles are already normalized
#print(imgs_swap.shape)
print(imgs.shape)

(16760,)
(16760, 160, 320, 3)


In [3]:
assert (steer_angle.shape[0] == imgs.shape[0]), "X (%d) and Y (%d) have unequal length." % (steer_angle.shape[0], imgs.shape[0])

In [4]:
drop_fraction = 0.5

# model start here
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5,
                 input_shape=(target_im_height, target_im_wide,3)))
model.add(Cropping2D(cropping=((60,20),(0,0)))) # cropping: tuple of tuple of int (length 2) 
                                                # How many units should be trimmed off at the 
                                                # beginning and end of the 2 cropping dimensions (width, height).
model.add(Convolution2D(24,5,5,
                        border_mode='valid', 
                        activation='relu', 
                        subsample=(2,2)))
model.add(Dropout(drop_fraction))
model.add(Convolution2D(36,5,5,
                        border_mode='valid', 
                        activation='relu', 
                        subsample=(2,2)))
model.add(Dropout(drop_fraction))
model.add(Convolution2D(48,5,5,
                        border_mode='valid',
                        activation='relu',
                        subsample=(2,2)))
model.add(Dropout(drop_fraction))
model.add(Convolution2D(64,3,3,
                        border_mode='valid', 
                        activation='relu', 
                        subsample=(1,1)))
model.add(Dropout(drop_fraction))
model.add(Convolution2D(64,3,3,
                        border_mode='valid', 
                        activation='relu', 
                        subsample=(1,1)))
model.add(Dropout(drop_fraction))
model.add(Flatten())
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 80, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 38, 158, 24)   1824        cropping2d_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 38, 158, 24)   0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [5]:
model.compile(loss='mse', 
              optimizer='adam', 
              metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(imgs, steer_angle, nb_epoch=10, validation_split=0.2, callbacks=[early_stopping])

Train on 13408 samples, validate on 3352 samples
Epoch 1/10
13408/13408 [==============================] - 313s - loss: 0.0321 - acc: 0.7423 - val_loss: 0.0498 - val_acc: 0.2810

In [6]:
model.save('model.h5')
# Save model to JSON
#with open('autopilot_basic_model.json', 'w') as outfile:
#    outfile.write(json.dumps(json.loads(model.to_json()), indent=2))

2017_02_11: First run of the model in the simulation.

2017_02_12: Recorded recovery but at low speed and high steering angle.  Ended up driving in to the lake.  
Cropped the images and scaled the steering angles of the recovery by 0.5 to reduce the oscillation and hopefully prevent driving off the bridge.  Too little correction.  Increased the scaling to 0.75 original.  Made it to the bridge and crashed in to the abutment.
Oops.  Cropping argument was wrong.
Added Dropout for each conv layer to prevent overfitting.  
Was able to complete laps of track 1.
Noticed the conversion from BGR to YUV in the model was not in drive.py.  Corrected drive.py.  Erratic driving and crashes off right side before bridge.
The I later noticed that the drive.py uses a different function to load the image. drive.py uses PIL.Image.open.  This seems to use RGB colorspace by default.  I adjusted the drive.py and found the car drove past bridge but driving erratic and, finally, off road.  I think it might be necessary to scale the steering angle down a bit again.
I scaled steering by /0.75 /2.  Retraining.